In [1]:
import numpy as np
import pandas as pd
from transformers import pipeline
from icecream import ic
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

In [2]:
reviews_raw = pd.read_csv("movie_data.csv")

In [3]:
reviews_raw

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0
...,...,...
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0


In [24]:
reviews_sample = reviews_raw.sample(n = 500)

In [5]:
reviews_sample.sentiment.value_counts()

sentiment
0    52
1    48
Name: count, dtype: int64

In [6]:
review1 = reviews_sample.iloc[2, 0]
print(review1)

Exactly what you would expect from a B-Movie. Deritive, gratuitous nudity, boring in parts, ridiculous gore and cheesy special-effects. Of course it could have been better, better acted (defintly) better written, directed, etc. But then I guess it wouldn't have been a B movie. The actors pretty much sucked, in fact this pretty much seemed like an episode of buffy the vampire slayer or something except with a lot of blood, profanity and nudity.<br /><br />Tiffany Shepis must be singled out. She absolutely is the scream queen of the new millennium. Not that acting really matters in these movies, but she was better than any of the other actors. She's also smokin hot, in that plastic jump suit thing she wore for the whole movie - wow! Her posterior is absolutely stunning in that outfit, I mean it every single time she turns around you can help but check her out. And near then end of the film the viewer is rewarded with seeing her completely nekked.<br /><br />So if your a looser b-movie ho

In [7]:
reviews_list = reviews_sample.iloc[:, 0].tolist()

In [8]:
classifier = pipeline("sentiment-analysis")
result = classifier(review1)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [9]:
result

[{'label': 'POSITIVE', 'score': 0.6249537467956543}]

In [10]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [11]:
# batch = tokenizer(reviews_list, padding = True, truncation = True, max_length = 512)

In [12]:
# with torch.no_grad():
#     outputs = model(**batch)
#     predictions = F.softmax(outputs.logits, dim = 1)
#     labels = torch.argmax(predictions, dim = 1)

In [13]:
# pred = [1 if i['label'] == 'POSITIVE' else 0 for i in result]
# pred

In [14]:
five_reviews = reviews_sample.iloc[:5, 0].tolist()

In [15]:
token = tokenizer(five_reviews, padding = True, truncation = True, max_length = 512, return_tensors = "pt")

In [16]:
token['input_ids']

tensor([[ 101, 1045, 3342,  ..., 1012, 1012,  102],
        [ 101, 1045, 2228,  ...,    0,    0,    0],
        [ 101, 3599, 2054,  ...,    0,    0,    0],
        [ 101, 2043, 1045,  ...,    0,    0,    0],
        [ 101, 1000, 1999,  ...,    0,    0,    0]])

In [18]:
# with torch.no_grad():
#     outputs = model(**token)
#     probs = F.softmax(outputs.logits, dim = 1)
#     pred = torch.argmax(probs, dim = 1)

In [17]:
# pred

In [19]:
# torch.sum(pred == torch.tensor(reviews_sample.iloc[:5, 1].tolist())).item()

In [25]:
correct = 0
for i in range(5, 501, 5):
    tokens = tokenizer(reviews_sample.iloc[i-5:i, 0].tolist(), padding = True, truncation = True, max_length = 512, return_tensors = "pt")
    with torch.no_grad():
        outputs = model(**tokens)
        probs = F.softmax(outputs.logits, dim = 1)
        pred = torch.argmax(probs, dim = 1)
        correct += torch.sum(pred == torch.tensor(reviews_sample.iloc[i-5:i, 1].tolist())).item()

In [26]:
acc = correct/500

In [27]:
correct

453

In [28]:
acc

0.906

#### Accuracy without fine tuning on 500 reviews = 90%